<a href="https://colab.research.google.com/github/robsonspereira/COVID19/blob/master/corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import plotly.express as px
import datetime
import matplotlib.pyplot as plt

In [2]:
d = datetime. date. today()
print(d)
day = '{:0>2}'.format(d.day)
month= '{:0>2}'.format(d.month)
year='{:0>4}'.format(d.year)
url = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-%s-%s-%s.xlsx"%(year,month,day)
print(url)


2020-03-23
https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-23.xlsx


In [3]:
base_import = pd.read_excel(url)
base_import.head()
base_import = base_import[base_import['Cases']!=0]
base_import.dropna(inplace=True)
base_import = base_import.sort_values(by =['GeoId','DateRep'])
base_import.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
100,2020-03-03,3,3,2020,1,0,Andorra,AD
99,2020-03-14,14,3,2020,1,0,Andorra,AD
98,2020-03-16,16,3,2020,3,0,Andorra,AD
97,2020-03-17,17,3,2020,9,0,Andorra,AD
95,2020-03-19,19,3,2020,39,0,Andorra,AD


In [4]:
Total_Cases = base_import[['GeoId','DateRep','Cases']]
Total_Cases['Total Cases']= Total_Cases.groupby(['GeoId']).cumsum()
Total_Cases.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,GeoId,DateRep,Cases,Total Cases
100,AD,2020-03-03,1,1
99,AD,2020-03-14,1,2
98,AD,2020-03-16,3,5
97,AD,2020-03-17,9,14
95,AD,2020-03-19,39,53


In [5]:
Total_Deaths = base_import[['GeoId','DateRep','Deaths']]
Total_Deaths['Total Deaths']= Total_Deaths.groupby(['GeoId']).cumsum()
Total_Deaths.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,GeoId,DateRep,Deaths,Total Deaths
100,AD,2020-03-03,0,0
99,AD,2020-03-14,0,0
98,AD,2020-03-16,0,0
97,AD,2020-03-17,0,0
95,AD,2020-03-19,0,0


In [6]:
GeoIDGroup = base_import.groupby(['GeoId']).agg({'DateRep':['min','max'],'Cases':['sum'],'Deaths':['sum']})
GeoIDGroup.columns = ['DateFirstReport','DateLastReport','Cases','Deaths'] 
GeoIDGroup['Duration'] = (GeoIDGroup['DateLastReport']-GeoIDGroup['DateFirstReport'])
GeoIDGroup['Duration'] = pd.to_numeric(GeoIDGroup['Duration'].dt.days, downcast='integer')
GeoIDGroup['CasesPerDay'] = (GeoIDGroup['Cases']/GeoIDGroup['Duration'].astype(int))
GeoIDGroup['DeathsPerCases'] = (GeoIDGroup['Deaths']/GeoIDGroup['Cases'])
GeoIDGroup

,DateFirstReport,DateLastReport,Cases,Deaths,Duration,CasesPerDay,DeathsPerCases
GeoId,,,,,,,
AD,2020-03-03,2020-03-23,113,0,20,5.650000,0.000000
AE,2020-01-27,2020-03-22,153,2,55,2.781818,0.013072
AF,2020-02-25,2020-03-23,34,0,27,1.259259,0.000000
AG,2020-03-15,2020-03-15,1,0,0,inf,0.000000
AL,2020-03-09,2020-03-23,89,2,14,6.357143,0.022472
...,...,...,...,...,...,...,...
VN,2020-01-24,2020-03-23,118,0,59,2.000000,0.000000
XK,2020-03-16,2020-03-23,31,1,7,4.428571,0.032258
ZA,2020-03-06,2020-03-23,274,0,17,16.117647,0.000000


In [7]:
GeoIDGroup.loc[['BR','IT','CN','US','ES','IE']]

,DateFirstReport,DateLastReport,Cases,Deaths,Duration,CasesPerDay,DeathsPerCases
GeoId,,,,,,,
BR,2020-02-26,2020-03-23,1546,25,26,59.461538,0.016171
IT,2020-01-31,2020-03-23,59138,5476,52,1137.269231,0.092597
CN,2019-12-31,2020-03-23,81649,3274,83,983.722892,0.040098
US,2020-01-21,2020-03-23,35206,471,62,567.838710,0.013378
ES,2020-02-01,2020-03-23,28572,1720,51,560.235294,0.060199
IE,2020-03-01,2020-03-23,906,4,22,41.181818,0.004415


In [8]:
  Events = base_import.merge(GeoIDGroup[['DateFirstReport']],left_on='GeoId',right_index=True, how ='left')
  Events['Day_report'] = (Events['DateRep']-Events['DateFirstReport']).dt.days.astype(int)
  Events= Events.merge(Total_Cases[['Total Cases']],left_index=True,right_index=True, how='left')
  Events= Events.merge(Total_Deaths[['Total Deaths']],left_index=True,right_index=True, how='left')
  Events

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,DateFirstReport,Day_report,Total Cases,Total Deaths
100,2020-03-03,3,3,2020,1,0,Andorra,AD,2020-03-03,0,1,0
99,2020-03-14,14,3,2020,1,0,Andorra,AD,2020-03-03,11,2,0
98,2020-03-16,16,3,2020,3,0,Andorra,AD,2020-03-03,13,5,0
97,2020-03-17,17,3,2020,9,0,Andorra,AD,2020-03-03,14,14,0
95,2020-03-19,19,3,2020,39,0,Andorra,AD,2020-03-03,16,53,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5340,2020-03-23,23,3,2020,34,0,South_Africa,ZA,2020-03-06,17,274,0
6364,2020-03-19,19,3,2020,2,0,Zambia,ZM,2020-03-19,0,2,0
6360,2020-03-23,23,3,2020,1,0,Zambia,ZM,2020-03-19,4,3,0
6367,2020-03-21,21,3,2020,1,0,Zimbabwe,ZW,2020-03-21,0,1,0


In [9]:
Events_filter=Events.query( 'Day_report<=100 ')
Events_filter= Events_filter.query('GeoId=="US" or GeoId=="IT" or GeoId=="CN" or GeoId=="BR"  or GeoId=="ES"')

graph = px.line(Events_filter,
                x = 'Day_report',
                y = 'Cases',
                color = 'GeoId',
                line_group ='GeoId',
                hover_name = 'GeoId')

graph.show()

In [10]:
graph = px.line(Events_filter,
                x = 'Day_report',
                y = 'Total Cases',
                color = 'GeoId',
                line_group ='GeoId',
                hover_name = 'Countries and territories')

graph.show()

In [11]:
graph = px.line(Events_filter,
                x = 'Day_report',
                y = 'Total Deaths',
                color = 'GeoId',
                line_group ='GeoId',
                hover_name = 'GeoId')

graph.show()

In [12]:
Events_today=Events.loc[Events['DateRep'] == d]
Events_today.sort_values(by =(['Cases']),ascending=False, inplace=True)
Events_today

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,DateFirstReport,Day_report,Total Cases,Total Deaths
6170,2020-03-23,23,3,2020,8459,131,United_States_of_America,US,2020-01-21,62,35206,471
3096,2020-03-23,23,3,2020,5560,649,Italy,IT,2020-01-31,52,59138,5476
5440,2020-03-23,23,3,2020,3646,394,Spain,ES,2020-02-01,51,28572,1720
2258,2020-03-23,23,3,2020,3311,27,Germany,DE,2020-01-28,55,24774,94
2072,2020-03-23,23,3,2020,1559,112,France,FR,2020-01-25,58,16018,673
...,...,...,...,...,...,...,...,...,...,...,...,...
4162,2020-03-23,23,3,2020,1,0,New_Caledonia,NC,2020-03-21,2,5,0
2156,2020-03-23,23,3,2020,1,0,French_Polynesia,PYF,2020-03-19,4,18,0
1989,2020-03-23,23,3,2020,1,0,Fiji,FJ,2020-03-20,3,3,0
2442,2020-03-23,23,3,2020,1,0,Grenada,GD,2020-03-23,0,1,0


In [13]:
Events_today_filter = Events_today.query('Cases >= 100')

graph = px.bar(Events_today_filter, x = 'GeoId', y = 'Cases')

#x=Events_today_filter['GeoId']
#y=Events_today_filter['Cases']
#plt.figure(figsize=(15,10))
#plt.bar(x,y)
#plt.title('Reports today: %s'%(d))
#plt.bar(x,y,align='center')
#plt.ylabel('Reports')

#plt.show()
graph.show()